#### Prepared by: Yerbol Aussat
#### CS-889. University of Waterloo



# Real-Time Finger Tracking

# Helper Functions

In [2]:
# Import necessary libraries and helper functions
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import cv2
import random
from helpers import imshow

In [3]:
# Method that generates mask that segments out skin regions 
def segment_skin_hsv(img):
    # Convert image to HSV
    hsv = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)

    # define range of hsv (for skin color)
    min_HSV = np.array([0, 58, 30], dtype = "uint8")
    max_HSV = np.array([33, 255, 255], dtype = "uint8")

    # create a mask and clean it up
    mask = cv2.inRange(hsv, min_HSV, max_HSV)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, None, iterations = 5)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, None, iterations = 5)
    mask = cv2.medianBlur(mask, 5)   
    
    return mask

In [11]:
def segment_skin_ycrcb(img):
    # Convert image to YCrCb
    imageYCrCb = cv2.cvtColor(img,cv2.COLOR_BGR2YCR_CB)    
    
    # Constants for finding range of skin color in YCrCb
    min_YCrCb = np.array([0,133,77],np.uint8)
    max_YCrCb = np.array([255,173,127],np.uint8)

    # Find region with skin tone in YCrCb image
    mask = cv2.inRange(imageYCrCb,min_YCrCb,max_YCrCb)
#     mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, None, iterations = 5)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, None, iterations = 3)
    mask = cv2.medianBlur(mask, 5)   
    
    return mask

In [4]:

def getHandContours(mask):
    # Get the contour of the hand and draw it on the original image
    im2, contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Calculate areas
    areas = [cv2.contourArea(cont) for cont in contours]
    
    # Only keep the largest contour
    if len(contours) == 0:
        return
    cnt = contours[np.argmax(areas)]
    return cnt

# Real-Time Finger Tracking Code

In [38]:
camera = cv2.VideoCapture(0)
# reduce frame size to speed it up
w = 640
camera.set(cv2.CAP_PROP_FRAME_WIDTH, w) 
camera.set(cv2.CAP_PROP_FRAME_HEIGHT, w * 3/4) 
camera.set(cv2.CAP_PROP_EXPOSURE,-4) 


while True:
    # Get frame and flip it
    ret, frame = camera.read()
    frame = cv2.flip(frame, 1)

    frame = cv2.medianBlur(frame,5) # Apply median blur
    mask = segment_skin_ycrcb(frame) # Mask
    cv2.imshow("MASK", mask)
    cnt = getHandContours(mask) # Hand Contour
    
    if cnt is None or len(cnt) <= 400:
        print "No contours found"

    else:        
        cnt2 = smoothenContour(cnt, 10)    
        cv2.drawContours(frame, [cnt], 0, (0,255,0), 2)
        
    if cv2.waitKey(5) == 27:
        break  
        
    cv2.imshow("VideoFrame", frame)

cv2.destroyAllWindows()
camera.release()


/usr/local/lib/python2.7/site-packages/scipy/interpolate/_fitpack_impl.py:226: RuntimeWarning: Setting x[0][1208]=x[0][0]
  (i, m, i)))
/usr/local/lib/python2.7/site-packages/scipy/interpolate/_fitpack_impl.py:226: RuntimeWarning: Setting x[0][744]=x[0][0]
  (i, m, i)))
